# Create CDAW CME Merge To LASCO Catalog
This notebook is for creating a merged CME/LASCO catalogs of both CME events and time intervals when there are no CMEs (depends on telescope). 

In [1]:
import requests
import pandas as pd

Need_To_Redownload = True

LASCO_Catalog_Filename = '../lasco_img_catalog_lvl_1.txt'
CME_Catalog_Filename = '../cme_event_catalog.txt'
Merged_Catalog_Filename = '../merged_cdaw_lasco_catalog.csv'

# sampling of non-events
C2_non_CME_Catalog_Filename = '../c2_non_events.csv'
C3_non_CME_Catalog_Filename = '../c3_non_events.csv'

In [2]:
# pull the lasco catalog to local disk
if Need_To_Redownload:
    
    # download LASCO catalog now
    lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/img_hdr.txt'
    response = requests.get(lasco_hdr_url, allow_redirects=True)
    
    # cache locally
    open(LASCO_Catalog_Filename, 'wb').write(response.content)


In [3]:
cme_data = pd.read_csv(CME_Catalog_Filename)
cme_data = cme_data.rename(columns={'Unnamed: 0':'CDAW_Id'})
cme_data

,CDAW_Id,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,Mass,Kinetic,MPA,Remarks,Duration_C2,Duration_C3,StartTime_C3_Delta
0,5,1996/01/31,06:52:13,274,47,158,219,99,0,-12.3*,2.5e+14*,3.2e+28*,272,Poor Event; Only 3 points,26582.278481,116518.987342,9746.835443
1,8,1996/02/12,05:47:26,91,53,160,100,211,236,2.0*,1.3e+15,1.6e+29,92,Poor Event,26250.000000,115062.500000,9625.000000
2,12,1996/03/02,04:11:53,88,36,108,95,120,177,1.0*,1.5e+14,8.7e+27,86,Poor Event,38888.888889,170462.962963,14259.259259
3,16,1996/03/07,07:51:56,91,43,60,60,61,75,0.1*,-------,-------,87,Poor Event,70000.000000,306833.333333,25666.666667
4,18,1996/03/09,04:27:49,84,39,93,102,82,0,-1.1*,-------,-------,82,Only 3 points,45161.290323,197956.989247,16559.139785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7610,30475,2020/08/29,00:00:05,86,16,281,220,339,478,8.4*,-------,-------,89,Poor Event,14946.619217,65516.014235,5480.427046
7611,30477,2020/08/29,15:36:05,284,11,196,227,163,0,-4.8*,-------,-------,282,Poor Event,21428.571429,93928.571429,7857.142857
7612,30478,2020/08/29,19:48:05,285,10,218,231,204,79,-2.0*,-------,-------,272,Poor Event,19266.055046,84449.541284,7064.220183
7613,30479,2020/08/30,01:25:44,258,42,117,0,213,286,3.5*,-------,-------,250,Poor Event,35897.435897,157350.427350,13162.393162


In [4]:
# now work to merge in information from LASCO img catalog
lasco_catalog = pd.read_fwf(LASCO_Catalog_Filename, index=False,
                            names=['filename', 'date-obs', 'time-obs','telescope', 'expTime',
                                   'NumCols', 'NumRows', 'StartCol', 'StartRow', 'Filter', 
                                   'Polarizer','Observ.Seq', 'LEB Program', 'junk'])

In [5]:
lasco_catalog

,filename,date-obs,time-obs,telescope,expTime,NumCols,NumRows,StartCol,StartRow,Filter,Polarizer,Observ.Seq,LEB Program,junk
0,12000001.fts,1995/12/08,02:05:12,C1,0.0,512,512,20,1,Ca XV,Clear,Dark,0.0,1.0
1,12000002.fts,1995/12/08,02:12:28,C1,1.9,512,512,20,1,Ca XV,Clear,Dark,0.0,1.0
2,22000001.fts,1995/12/08,02:15:46,C2,0.0,512,512,20,1,Blue,Clear,Dark,0.0,1.0
3,22000002.fts,1995/12/08,02:19:24,C2,1.9,512,512,20,1,Blue,Clear,Dark,0.0,1.0
4,32000001.fts,1995/12/08,02:21:56,C3,0.0,512,512,20,1,Clear,Clear,Dark,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177593,32647530.fts,2020/12/31,23:18:06,C3,17.6,1024,1024,20,1,Clear,Clear,Normal,0.0,0.0
2177594,22797929.fts,2020/12/31,23:24:05,C2,25.1,1024,1024,20,1,Orange,Clear,Normal,0.0,9.0
2177595,32647531.fts,2020/12/31,23:30:05,C3,17.6,1024,1024,20,1,Clear,Clear,Normal,0.0,0.0
2177596,22797930.fts,2020/12/31,23:36:05,C2,25.2,1024,1024,20,1,Orange,Clear,Normal,0.0,9.0


In [6]:
# Drop C1, EIT observations (this speeds up datetime calc below)
lasco_catalog = lasco_catalog[lasco_catalog['telescope'] != 'C1']
lasco_catalog = lasco_catalog[lasco_catalog['telescope'] != 'EIT']

#date_filt = lasco_catalog[lasco_catalog['date-obs']=='2020/05/28']
#date_filt[date_filt['time-obs']=='21:12:11']
len(lasco_catalog)

1445817

In [7]:
from datetime import datetime, date, time, timezone, timedelta

# add column of python datetime for easier handling
datetime_col = []
for row in lasco_catalog.iterrows():
    date_obs = row[1]['date-obs']
    time_obs = row[1]['time-obs']
    
    date_obs = [int(t) for t in row[1]['date-obs'].split('/')]
    time_obs = [int(t) for t in row[1]['time-obs'].split(':')]
    
    datetime_col.append(datetime(year=date_obs[0], month=date_obs[1], day=date_obs[2], hour=time_obs[0], minute=time_obs[1], second=time_obs[2]))

lasco_catalog['datetime'] = datetime_col
lasco_catalog

,filename,date-obs,time-obs,telescope,expTime,NumCols,NumRows,StartCol,StartRow,Filter,Polarizer,Observ.Seq,LEB Program,junk,datetime
2,22000001.fts,1995/12/08,02:15:46,C2,0.0,512,512,20,1,Blue,Clear,Dark,0.0,1.0,1995-12-08 02:15:46
3,22000002.fts,1995/12/08,02:19:24,C2,1.9,512,512,20,1,Blue,Clear,Dark,0.0,1.0,1995-12-08 02:19:24
4,32000001.fts,1995/12/08,02:21:56,C3,0.0,512,512,20,1,Clear,Clear,Dark,0.0,1.0,1995-12-08 02:21:56
5,32000002.fts,1995/12/08,02:25:57,C3,1.9,512,512,20,1,Clear,Clear,Dark,0.0,1.0,1995-12-08 02:25:57
7,22000003.fts,1995/12/08,21:43:48,C2,2.9,1024,1024,20,1,Lens,Clear,Lamp,0.0,0.0,1995-12-08 21:43:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177593,32647530.fts,2020/12/31,23:18:06,C3,17.6,1024,1024,20,1,Clear,Clear,Normal,0.0,0.0,2020-12-31 23:18:06
2177594,22797929.fts,2020/12/31,23:24:05,C2,25.1,1024,1024,20,1,Orange,Clear,Normal,0.0,9.0,2020-12-31 23:24:05
2177595,32647531.fts,2020/12/31,23:30:05,C3,17.6,1024,1024,20,1,Clear,Clear,Normal,0.0,0.0,2020-12-31 23:30:05
2177596,22797930.fts,2020/12/31,23:36:05,C2,25.2,1024,1024,20,1,Orange,Clear,Normal,0.0,9.0,2020-12-31 23:36:05


In [8]:
# split up catalog by C2 and C3
c2_lasco = lasco_catalog[lasco_catalog['telescope']=='C2']
c3_lasco = lasco_catalog[lasco_catalog['telescope']=='C3']

In [9]:
# Do the merge of catalogs, using CME events for lookup of filename in LASCO.
# We are not just getting CME images, howerver. For purposes of background subtraction we need to 
# capture the image before first CME event image

cme_ids=[]
filenames=[]
filters = []
dates=[]
times=[]

lasco_keepCols = ['filename', 'telescope', 'NumCols', 'NumRows', 'Filter', 'Polarizer', 'datetime', 'expTime', 'date-obs']
cdaw_keepCols = ['Remarks', 'Central', 'Width', 'Linear', 'CDAW_Id']
bkgnd_keepCols = ['filename', 'NumCols', 'NumRows', 'Filter', 'datetime', 'expTime', 'Polarizer']
bckgnd_fileinfo = [ 'bkgnd_'+colname for colname in bkgnd_keepCols ]

merged_data = {  col:[] for col in lasco_keepCols + cdaw_keepCols + bckgnd_fileinfo }

for event in cme_data.iterrows():
    
    # grab date-time-duration info for our event
    cme_start_date = [int(t) for t in event[1]['Date'].split('/')]
    cme_start_time = [int(t) for t in event[1]['Time'].split(':')]
    cme_c3_delta = timedelta(seconds=event[1]['StartTime_C3_Delta'])
    cme_c2_duration = timedelta(seconds=float(event[1]['Duration_C2']))
    cme_c3_duration = timedelta(seconds=float(event[1]['Duration_C3']))
    
    # convert to Python datetime for easier slicing
    cme_date = date (year=cme_start_date[0], month=cme_start_date[1], day=cme_start_date[2])
    cme_time = time(hour=cme_start_time[0],minute=cme_start_time[1],second=cme_start_time[2])
    
    c2_start = datetime.combine(cme_date, cme_time)
    c2_end   = c2_start + cme_c2_duration
    
    c3_start = c2_start + cme_c3_delta
    c3_end   = c3_start + cme_c3_duration
    
    # get C2 entries
    
    # capture prior image, store with cme image id
    previous_c2_image = c2_lasco[c2_lasco['datetime']<c2_start].iloc[-1]
    
    # and a slice of all of the images which follow
    c2_slice = c2_lasco[c2_lasco['datetime']>c2_start]
    for lasco_row in c2_slice[c2_slice['datetime']<=c2_end].iterrows():
        
        for col in lasco_keepCols:
            merged_data[col].append(lasco_row[1][col])
          
        for col in cdaw_keepCols:
            merged_data[col].append(event[1][col])
        
        #  add in background file to use
        for col in bkgnd_keepCols: 
            merged_data['bkgnd_'+col].append(previous_c2_image[col])
    

    # get C3 entries
    
    # capture prior image, store with cme image id
    previous_c3_image = c3_lasco[c3_lasco['datetime']<c3_start].iloc[-1]
    
    c3_slice = c3_lasco[c3_lasco['datetime']>c3_start]
    for lasco_row in c3_slice[c3_slice['datetime']<=c3_end].iterrows():
        
        for col in lasco_keepCols:
            merged_data[col].append(lasco_row[1][col])
          
        for col in cdaw_keepCols:
            merged_data[col].append(event[1][col])
         
        #  add in background file to use
        for col in bkgnd_keepCols: 
            merged_data['bkgnd_'+col].append(previous_c3_image[col])
    
# convert results to DataFrame
result = pd.DataFrame(merged_data)
result

,filename,telescope,NumCols,NumRows,Filter,Polarizer,datetime,expTime,date-obs,Remarks,...,Width,Linear,CDAW_Id,bkgnd_filename,bkgnd_NumCols,bkgnd_NumRows,bkgnd_Filter,bkgnd_datetime,bkgnd_expTime,bkgnd_Polarizer
0,22000294.fts,C2,1024,1024,Orange,Clear,1996-01-31 08:01:38,25.4,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
1,22000295.fts,C2,1024,1024,Orange,Clear,1996-01-31 09:11:14,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
2,22000296.fts,C2,1024,1024,Orange,Clear,1996-01-31 10:20:39,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
3,22000297.fts,C2,1024,1024,Orange,Clear,1996-01-31 11:30:19,25.9,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
4,22000298.fts,C2,1024,1024,Orange,Clear,1996-01-31 12:40:17,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574731,32633227.fts,C3,1024,1024,Clear,Clear,2020-09-01 03:18:05,17.6,2020/09/01,Poor Event,...,53,213,30480,32645092.fts,1024,1024,Clear,2019-10-30 23:42:05,17.9,Clear
574732,32633228.fts,C3,1024,1024,Clear,Clear,2020-09-01 03:30:05,17.8,2020/09/01,Poor Event,...,53,213,30480,32645092.fts,1024,1024,Clear,2019-10-30 23:42:05,17.9,Clear
574733,32633229.fts,C3,1024,1024,Clear,Clear,2020-09-01 03:42:05,17.6,2020/09/01,Poor Event,...,53,213,30480,32645092.fts,1024,1024,Clear,2019-10-30 23:42:05,17.9,Clear
574734,32633230.fts,C3,1024,1024,Clear,Clear,2020-09-01 03:54:07,17.6,2020/09/01,Poor Event,...,53,213,30480,32645092.fts,1024,1024,Clear,2019-10-30 23:42:05,17.9,Clear


In [10]:
# save the merged catalog
result.to_csv(Merged_Catalog_Filename)

In [11]:
# test it out, try to pull a few intervals for events
C2_data = result[result['telescope']=='C2']
C2_data[:10]

,filename,telescope,NumCols,NumRows,Filter,Polarizer,datetime,expTime,date-obs,Remarks,...,Width,Linear,CDAW_Id,bkgnd_filename,bkgnd_NumCols,bkgnd_NumRows,bkgnd_Filter,bkgnd_datetime,bkgnd_expTime,bkgnd_Polarizer
0,22000294.fts,C2,1024,1024,Orange,Clear,1996-01-31 08:01:38,25.4,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
1,22000295.fts,C2,1024,1024,Orange,Clear,1996-01-31 09:11:14,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
2,22000296.fts,C2,1024,1024,Orange,Clear,1996-01-31 10:20:39,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
3,22000297.fts,C2,1024,1024,Orange,Clear,1996-01-31 11:30:19,25.9,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
4,22000298.fts,C2,1024,1024,Orange,Clear,1996-01-31 12:40:17,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
5,22000299.fts,C2,1024,1024,Orange,Clear,1996-01-31 13:50:01,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
26,22000487.fts,C2,1024,1024,Orange,Clear,1996-02-12 07:01:57,25.1,1996/02/12,Poor Event,...,53,160,8,22000485.fts,1024,1024,Orange,1996-02-12 04:32:50,25.1,Clear
27,22000488.fts,C2,1024,1024,Orange,Clear,1996-02-12 08:16:27,25.1,1996/02/12,Poor Event,...,53,160,8,22000485.fts,1024,1024,Orange,1996-02-12 04:32:50,25.1,Clear
28,22000489.fts,C2,1024,1024,Orange,Clear,1996-02-12 09:30:48,25.1,1996/02/12,Poor Event,...,53,160,8,22000485.fts,1024,1024,Orange,1996-02-12 04:32:50,25.1,Clear
29,22000490.fts,C2,1024,1024,Orange,Clear,1996-02-12 10:45:46,25.1,1996/02/12,Poor Event,...,53,160,8,22000485.fts,1024,1024,Orange,1996-02-12 04:32:50,25.1,Clear


In [12]:
#c2_lasco

# now build out the catalog of NON-CME times

In [13]:
# now build out the catalog of NON-CME times
keepCols = ['filename', 'datetime', 'date-obs', 'NumCols', 'NumRows', 'Filter', 'Polarizer', 'expTime']
c2_intervals = c2_lasco[keepCols]
c2_intervals.set_index('datetime')

c3_intervals = c3_lasco[keepCols]
c3_intervals.set_index('datetime')

c2_intervals

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
2,22000001.fts,1995-12-08 02:15:46,1995/12/08,512,512,Blue,Clear,0.0
3,22000002.fts,1995-12-08 02:19:24,1995/12/08,512,512,Blue,Clear,1.9
7,22000003.fts,1995-12-08 21:43:48,1995/12/08,1024,1024,Lens,Clear,2.9
8,22000004.fts,1995-12-08 22:04:07,1995/12/08,1024,1024,Lens,Clear,4.5
28,22000005.fts,1995-12-11 19:55:05,1995/12/11,1024,1024,Blue,Clear,2.3
...,...,...,...,...,...,...,...,...
2177584,22797926.fts,2020-12-31 22:24:05,2020/12/31,1024,1024,Orange,Clear,25.2
2177586,22797927.fts,2020-12-31 22:36:05,2020/12/31,1024,1024,Orange,Clear,25.1
2177592,22797928.fts,2020-12-31 23:12:12,2020/12/31,1024,1024,Orange,Clear,25.1
2177594,22797929.fts,2020-12-31 23:24:05,2020/12/31,1024,1024,Orange,Clear,25.1


In [14]:
# step through CDAW catalog and calculate when there is NOT an event
    
c2_non_events = { k:[] for k in c2_intervals.columns }
c3_non_events = { k:[] for k in c3_intervals.columns }

sformat = "%Y-%m-%d %H:%M:%S"
    
last_c2_end = None
last_c3_end = None

for event in cme_data.iterrows():

    # grab date-time-duration info for our event
    cme_start_date = [int(t) for t in event[1]['Date'].split('/')]
    cme_start_time = [int(t) for t in event[1]['Time'].split(':')]
    cme_c3_delta = timedelta(seconds=event[1]['StartTime_C3_Delta'])
    cme_c2_duration = timedelta(seconds=float(event[1]['Duration_C2']))
    cme_c3_duration = timedelta(seconds=float(event[1]['Duration_C3']))

    # convert to Python datetime for easier slicing
    cme_date = date (year=cme_start_date[0], month=cme_start_date[1], day=cme_start_date[2])
    cme_time = time(hour=cme_start_time[0],minute=cme_start_time[1],second=cme_start_time[2])

    c2_start = datetime.combine(cme_date,cme_time)
    c2_end   = c2_start + cme_c2_duration

    c3_start = c2_start + cme_c3_delta
    c3_end   = c3_start + cme_c3_duration

    # we have free interval for C2, C3 based on prior event end and 
    # the start of the new event

    if last_c2_end != None:
        for c2_ne in c2_intervals.loc[last_c2_end.strftime(sformat):c2_start.strftime(sformat)].iterrows():
            for col in c2_intervals.columns:
                c2_non_events[col].append(c2_ne[1][col])
                
    if last_c3_end != None:
        for c3_ne in c3_intervals.loc[last_c3_end.strftime(sformat):c3_start.strftime(sformat)].iterrows():
            for col in c3_intervals.columns:
                c3_non_events[col].append(c3_ne[1][col])

    last_c2_end = c2_end
    last_c3_end = c3_end

In [15]:
c2_non_events = pd.DataFrame(c2_non_events)
c2_non_events

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
0,22706303.fts,2018-12-17 00:12:05,2018/12/17,1024,1024,Orange,Clear,25.3
1,22706304.fts,2018-12-17 00:24:05,2018/12/17,1024,1024,Orange,Clear,25.1
2,22706305.fts,2018-12-17 00:36:05,2018/12/17,1024,1024,Orange,Clear,25.1
3,22706306.fts,2018-12-17 00:48:05,2018/12/17,1024,1024,Orange,Clear,26.1
4,22706307.fts,2018-12-17 01:25:42,2018/12/17,1024,1024,Orange,Clear,25.1
...,...,...,...,...,...,...,...,...
9586,22717913.fts,2019-03-22 06:12:11,2019/03/22,1024,1024,Orange,Clear,25.3
9587,22717914.fts,2019-03-22 06:24:05,2019/03/22,1024,1024,Orange,Clear,25.1
9588,22717915.fts,2019-03-22 06:36:05,2019/03/22,1024,1024,Orange,Clear,25.9
9589,22717916.fts,2019-03-22 07:12:06,2019/03/22,1024,1024,Orange,Clear,25.1


In [16]:
c3_non_events = pd.DataFrame(c3_non_events)
c3_non_events

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
0,32564733.fts,2018-12-12 21:30:06,2018/12/12,1024,1024,Clear,Clear,17.6
1,32564734.fts,2018-12-12 21:42:05,2018/12/12,1024,1024,Clear,Clear,17.6
2,32564735.fts,2018-12-12 21:54:06,2018/12/12,1024,1024,Clear,Clear,18.3
3,32564736.fts,2018-12-12 22:06:05,2018/12/12,1024,1024,Clear,Clear,17.8
4,32564737.fts,2018-12-12 22:18:05,2018/12/12,1024,1024,Clear,Clear,17.6
...,...,...,...,...,...,...,...,...
7325,32575273.fts,2019-03-18 03:42:05,2019/03/18,1024,1024,Clear,Clear,17.6
7326,32575274.fts,2019-03-18 03:54:06,2019/03/18,1024,1024,Clear,Clear,17.6
7327,32575275.fts,2019-03-18 04:06:05,2019/03/18,1024,1024,Clear,Clear,17.7
7328,32575276.fts,2019-03-18 04:18:05,2019/03/18,1024,1024,Clear,Clear,17.9


In [17]:
# save it 
c2_non_events.to_csv(C2_non_CME_Catalog_Filename, index=False)
c3_non_events.to_csv(C3_non_CME_Catalog_Filename, index=False)

In [18]:
event_catalog  = result
test_time = event_catalog['datetime'][1]
test_time

Timestamp('1996-01-31 09:11:14')

In [19]:
len(event_catalog), len(c2_intervals)

(574736, 798112)

In [20]:
# build out a filtered catlog so that we match only stuff we can accept
# for differencing later on (e.g. same params as before)
c2_a = c2_intervals[c2_intervals['expTime']>24.]
c2_b = c2_a[c2_a['expTime']<27.]
c2_c = c2_b[c2_b['Filter'].str.match('Orang')]
c2_d = c2_c[c2_c['NumCols'] == 1024]
c2_filt_cat = c2_d[c2_d['NumRows'] == 1024]

c2_filt_cat

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
4310,22000254.fts,1996-01-28 17:18:10,1996/01/28,1024,1024,Orange,Clear,25.1
4479,22000278.fts,1996-01-30 03:54:18,1996/01/30,1024,1024,Orange,Clear,25.0
4483,22000279.fts,1996-01-30 04:38:27,1996/01/30,1024,1024,Orange,Clear,25.1
4487,22000280.fts,1996-01-30 05:56:07,1996/01/30,1024,1024,Orange,Clear,25.1
4491,22000281.fts,1996-01-30 07:13:53,1996/01/30,1024,1024,Orange,Clear,25.1
...,...,...,...,...,...,...,...,...
2177584,22797926.fts,2020-12-31 22:24:05,2020/12/31,1024,1024,Orange,Clear,25.2
2177586,22797927.fts,2020-12-31 22:36:05,2020/12/31,1024,1024,Orange,Clear,25.1
2177592,22797928.fts,2020-12-31 23:12:12,2020/12/31,1024,1024,Orange,Clear,25.1
2177594,22797929.fts,2020-12-31 23:24:05,2020/12/31,1024,1024,Orange,Clear,25.1


In [21]:
d = c2_filt_cat[c2_filt_cat['datetime']<test_time]
foo = d.iloc[0]
type(foo), foo

(pandas.core.series.Series,
 filename            22000254.fts
 datetime     1996-01-28 17:18:10
 date-obs              1996/01/28
 NumCols                     1024
 NumRows                     1024
 Filter                    Orange
 Polarizer                  Clear
 expTime                     25.1
 Name: 4310, dtype: object)